In [1]:
from pyspark.sql import SparkSession



spark = SparkSession.builder \
.appName('Table Demo') \
.enableHiveSupport() \
.getOrCreate()

25/02/05 07:35:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
df = spark.read \
.format ('csv') \
.option('header','True') \
.option('inferSchema','True') \
.load('/tmp/customers_100.csv')

In [3]:
df.show(5)

+-----------+----------+---------+-----------+-------+-------------------+---------+
|customer_id|      name|     city|      state|country|  registration_date|is_active|
+-----------+----------+---------+-----------+-------+-------------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|2023-06-29 00:00:00|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|2023-12-07 00:00:00|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|2023-10-27 00:00:00|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|2023-10-17 00:00:00|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|2023-03-14 00:00:00|    false|
+-----------+----------+---------+-----------+-------+-------------------+---------+
only showing top 5 rows



In [9]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [8]:
spark.sql('drop table default.customers_persistent')

DataFrame[]

In [44]:
df.createOrReplaceTempView('temp_customers')


In [11]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [14]:
spark.sql('select * from temp_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-------------------+---------+
|customer_id|      name|     city|      state|country|  registration_date|is_active|
+-----------+----------+---------+-----------+-------+-------------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|2023-06-29 00:00:00|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|2023-12-07 00:00:00|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|2023-10-27 00:00:00|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|2023-10-17 00:00:00|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|2023-03-14 00:00:00|    false|
+-----------+----------+---------+-----------+-------+-------------------+---------+



In [19]:
df.createTempView('temp_customers')

AnalysisException: Temporary view 'temp_customers' already exists

# Global Temporary Table

In [23]:
df.createOrReplaceGlobalTempView('global_customers')

In [24]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [28]:
spark.sql('show tables in global_temp').show()

+-----------+----------------+-----------+
|  namespace|       tableName|isTemporary|
+-----------+----------------+-----------+
|global_temp|global_customers|       true|
|           |  temp_customers|       true|
+-----------+----------------+-----------+



In [26]:
spark.sql('select * from global_temp.global_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-------------------+---------+
|customer_id|      name|     city|      state|country|  registration_date|is_active|
+-----------+----------+---------+-----------+-------+-------------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|2023-06-29 00:00:00|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|2023-12-07 00:00:00|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|2023-10-27 00:00:00|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|2023-10-17 00:00:00|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|2023-03-14 00:00:00|    false|
+-----------+----------+---------+-----------+-------+-------------------+---------+



In [27]:
spark.sql('drop table global_temp.temp_customers')

DataFrame[]

# Persistent Table

In [33]:
df.write.mode('overwrite').saveAsTable('customers_persistent')

25/02/05 08:11:11 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [34]:
spark.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
|         |      temp_customers|       true|
+---------+--------------------+-----------+



In [15]:
spark_new = spark.newSession()

In [17]:
spark_new.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [18]:
spark_new.sql('select * from temp_customers limit 5').show()

AnalysisException: Table or view not found: temp_customers; line 1 pos 14;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Project [*]
      +- 'UnresolvedRelation [temp_customers], [], false


# global temp

In [30]:
spark_new.sql('show tables in global_temp').show()

+-----------+----------------+-----------+
|  namespace|       tableName|isTemporary|
+-----------+----------------+-----------+
|global_temp|global_customers|       true|
+-----------+----------------+-----------+



In [31]:
spark_new.sql('select * from global_temp.global_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-------------------+---------+
|customer_id|      name|     city|      state|country|  registration_date|is_active|
+-----------+----------+---------+-----------+-------+-------------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|2023-06-29 00:00:00|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|2023-12-07 00:00:00|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|2023-10-27 00:00:00|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|2023-10-17 00:00:00|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|2023-03-14 00:00:00|    false|
+-----------+----------+---------+-----------+-------+-------------------+---------+



In [35]:
spark_new.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
+---------+--------------------+-----------+



In [38]:
spark.sparkContext.applicationId

'application_1738733427052_0010'

In [39]:
spark_new.sparkContext.applicationId

'application_1738733427052_0010'

In [41]:
spark_new.stop()

In [43]:
spark.stop()